In [1]:
# code is based from: https://github.com/ALIDmed/indeed_jobs_scraper/blob/main/main.py
# credit ALIDmed for the setup, but made some changes for my needs & additional optimizations as well

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager as CM
from selenium.common.exceptions import NoSuchElementException
# import random
# from time import sleep
import pandas as pd


indeed = {
    'job_input': '//*[@id="text-input-what"]',
    'location_input': '//*[@id="text-input-where"]',
    'find_jobs': "//button[contains(text(),'Search')]",
    'jobs': '//*[@id="mosaic-provider-jobcards"]/ul/li',
    'salary': './/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[3]/div/span',
    'title': './/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[1]',
    'rating': './/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/span[2]',
    'job_location': './/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/div[1]',
    'company': './/div/div[1]/div/div[1]/div/table[1]/tbody/tr/td/div[2]/span[1]',
    'next_page': '//a[@data-testid="pagination-page-next"]'
}


JOB = 'software engineer'
LOCATION = 'United States'
MAX_JOBS = 3000


options = Options()
options.add_experimental_option("detach", True)
options.add_argument("--disable-notifications")
options.add_argument("--blink-settings=imagesEnabled=false")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36")
options.add_argument("--headless")


services = Service(executable_path=CM().install())

driver = webdriver.Chrome(service=services, options=options)
driver.maximize_window()
driver.implicitly_wait(1.5)

indeed_link = 'https://www.indeed.com/'

driver.get(indeed_link)

job_input = driver.find_element(by='xpath' , value=indeed['job_input'])
location_input = driver.find_element(by='xpath' , value=indeed['location_input'])
find_jobs = driver.find_element(by='xpath' , value=indeed['find_jobs'])

job_input.send_keys(JOB)

# Clear the location input field
location_input.send_keys(Keys.CONTROL + "a")
location_input.send_keys(Keys.DELETE)

location_input.send_keys(LOCATION)
find_jobs.click()


data = {
    'title': [],
    'salary': [],
    'company': [],
    'rating': [],
    'job_location': [],
}

while len(data['title']) < MAX_JOBS:
    
    jobs = driver.find_elements(by='xpath', value=indeed['jobs'])
    
    try:
        close_button = driver.find_element(by='xpath', value='//button[@class="icl-CloseButton icl-Modal-close"]')
        close_button.click()
    except NoSuchElementException:
        pass
    
    for job in jobs:

        if len(data['title']) >= MAX_JOBS:
            break

        # job.location_once_scrolled_into_view
        # job.click()
        
        for key in ['title', 'salary', 'company', 'rating', 'job_location']:
            try:
                info = job.find_element(by='xpath', value=indeed[key]).text
                data[key].append(info)
                # print(f"{key}: {info}")
            except NoSuchElementException:
                data[key].append('NaN')
            
    try:
        next_page_button = driver.find_element(by='xpath', value=indeed['next_page'])
        next_page_button.click()
    except NoSuchElementException:
        print('Reached the last page')
        break

driver.quit()

df = pd.DataFrame(data=data)
df.to_csv('job_data.csv', index=False)

Reached the last page
